In [1]:
import skopt
from smallCombinerBigMagnets import *
import time
t=time.time()
swarm,lattice=compute_Sol(1e-5, 20.0,2000,50)
print(time.time()-t) #58

0.0084375 0.0045000000000000005 4.255942350631103 4.255942350631103
12.007129907608032


In [41]:
import scipy.interpolate as spi
class PhaseSpaceInterpolater:
    def __init__(self,X,Y):
        #X: evaluation points array with shape n,m where n is number of points, and m is dimension
        #Y: values
        self.X=X
        self.Y=Y
        self.xBounds=[] #list of tuples (min,max) for x
        
        self.xNormalized=np.zeros(X.shape)
        for i in range(X.shape[1]): #loop through dimensions
            self.xBounds.append((X[:,i].min(),X[:,i].max()))
            xRenormed=(X[:,i]-self.xBounds[i][0])/(self.xBounds[i][1]-self.xBounds[i][0])
            self.xNormalized[:, i]=xRenormed
        self.interpolater=spi.NearestNDInterpolator(self.xNormalized, self.Y)
    def normalize(self,args):
        #x array to be normalized
        argsNew=[]
        i=0
        for arg in args:
            argNorm=(arg-self.xBounds[i][0])/(self.xBounds[i][1]-self.xBounds[i][0])
            #if argNorm>1 or argNorm<0:
            #    print('input',args)
            #    print('bounds',self.xBounds)
            #    raise Exception('OUT OF BOUNDS')
            argsNew.append(argNorm)
            i+=1
        return argsNew
    def __call__(self,*args):
        val=self.interpolater(self.normalize(args))
        if val[0] is None:
            print('outside the region')
            print(args)
            return np.nan
        else:
            return val[0]

In [42]:
phaseSpacePoints=[] #holds the coordinates of the particles in phase space at the combiner output
revolutions=[] #values of revolution for each particle in swarm


for i in range(swarm[0].num_Particles()):
    q = swarm[0].particles[i].q.copy()[1:] #only y and z. x is the same
    p = swarm[0].particles[i].p[:].copy()
    Xi = np.append(q, p)  # phase space coords are 2 position values and 3 momentum
    phaseSpacePoints.append(Xi)
    revolutions.append(swarm[2].particles[i].revolutions)
fitFunc = PhaseSpaceInterpolater(np.asarray(phaseSpacePoints), revolutions)

[(-0.008420542384955813, 0.008436781933500411), (-0.004492508972203909, 0.004496904353450389), (-200.9995229292769, -199.00062797142093), (-4.253320100725664, 4.252265988596445), (-4.254955441715037, 4.2544898037100936)]


In [51]:
(1/10000)**(.2)

0.15848931924611134

In [47]:
(2000**.2)**2

20.91279105182547

15625